In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('inputs/train.csv', index_col='id')
data['date'] = pd.to_datetime(data['date'])

stores = pd.read_csv('inputs/stores.csv')
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [2]:
data_stores = data.merge(stores, left_on='store_nbr', right_on='store_nbr')

In [3]:
transact = pd.read_csv('inputs/transactions.csv')
transact['date'] = pd.to_datetime(transact['date'])
data_stores_trans  = data_stores.merge(transact, on=['store_nbr', 'date'])

In [4]:
data_stores_trans.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions
0,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13,2111
1,2013-01-02,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,2111
2,2013-01-02,1,BEAUTY,2.0,0,Quito,Pichincha,D,13,2111
3,2013-01-02,1,BEVERAGES,1091.0,0,Quito,Pichincha,D,13,2111
4,2013-01-02,1,BOOKS,0.0,0,Quito,Pichincha,D,13,2111


In [5]:
df = data_stores_trans.copy()

In [6]:
# df['sales'].replace(0, np.nan, inplace=True)

In [7]:
# df.groupby(['store_nbr']).median().sort_values('transactions')

THE CODE BELOW INTEGRATES THE HW4 STUFF AND ADDS THE WORK FROM ABOVE...

In [8]:
# path = 'inputs/'
# # read oil price
# data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
#
# ########################################################################################################################
# # TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# # Hint: check the documentation of .rolling() method of pandas.DataFrame
# ########################################################################################################################
# data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(7).mean()
#
# # Create continguous moving average of oil prices
# calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))
#
# ########################################################################################################################
# # TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# # as calendar.
# # TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# # Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
# ########################################################################################################################
# calendar = calendar.join(data_oil)
#
# data_oil['ma_oil'].fillna(method='ffill', inplace=True)
# calendar['ma_oil'].fillna(method='ffill', inplace=True)

In [9]:
data = pd.read_csv('inputs/train.csv', index_col='id')
data['date'] = pd.to_datetime(data['date'])
data['nat_holiday'] = 0
data['reg_holiday'] = 0
data['loc_holiday'] = 0

stores = pd.read_csv('inputs/stores.csv', index_col='store_nbr')

holidays = pd.read_csv('inputs/holidays_events.csv')
holidays['date'] = pd.to_datetime(holidays['date'])

holidays = holidays.reset_index()
stores = stores.reset_index()

for index, row in holidays.iterrows():
    store_nbr_list = []
    if row['locale'] == 'Local':
        for sindex, srow in stores.iterrows():
            if srow['city'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'loc_holiday'] = 1
    if row['locale'] == 'Regional':
        for sindex, srow in stores.iterrows():
            if srow['state'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'reg_holiday'] = 1
    if row['locale'] == 'National':
        data.loc[data['date'] == row['date'], 'nat_holiday'] = 1

In [10]:
path = 'inputs/'
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True)

########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(7).mean()

# Create continguous moving average of oil prices
calendar = pd.DataFrame()
calendar['date'] = pd.date_range('2013-01-01', '2017-08-31')
# calendar.head()
########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
cally = pd.merge(calendar, data_oil, how='outer', on='date' )
calendar = cally.copy()
# calendar = calendar.merge(data_oil, on='date')

# calendar.head()

data_oil['ma_oil'].fillna(method='ffill', inplace=True)
calendar['ma_oil'].fillna(method='ffill', inplace=True)

In [11]:
calendar['wd'] = [i.dayofweek < 5 for i in calendar['date']]

In [12]:
stores_new = pd.read_csv(path + 'stores.csv',
                         usecols=['store_nbr', 'city', 'state', 'type', 'cluster'],
                         dtype={'store_nbr': 'category', 'city': 'category', 'state': 'category', 'type': 'category', 'cluster': 'category'})

In [13]:
transactions = pd.read_csv(path + 'transactions.csv',
                          usecols=['store_nbr', 'date', 'transactions'],
                          dtype={'store_nbr': 'category', 'transactions': 'float32'},
                          parse_dates=['date'], infer_datetime_format=True)
# transactions.date = transactions.date.dt.to_period('D')

In [14]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32', 'onpromotion': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

# df_train.date = df_train.date.dt.to_period('D')
# df_train = temp.merge(transactions, on=['store_nbr','date'])


In [15]:
# temp = df_train.merge(transactions, on =['store_nbr', 'date'])
temp = pd.merge(transactions, df_train, how='outer', on =['store_nbr', 'date'])

In [16]:
temp2 = temp.merge(stores_new, on='store_nbr')

In [17]:
temp2.shape

(3000888, 10)

In [18]:
df_train = temp2.copy()
# df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

In [19]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

# df_test.date = df_test.date.dt.to_period('D')
# Adding in additional features


df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [20]:
store1 = df_train.loc[df_train['store_nbr'] == '1']
# df_train.head()
# df_train.groupby('store_nbr')
# ddf = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

In [21]:
testy = df_train.merge(calendar, on='date')
testy

,date,store_nbr,transactions,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,ma_oil,wd
0,2013-01-01,25,770.0,AUTOMOTIVE,0.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True
1,2013-01-01,25,770.0,BABY CARE,0.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True
2,2013-01-01,25,770.0,BEAUTY,2.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True
3,2013-01-01,25,770.0,BEVERAGES,810.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True
4,2013-01-01,25,770.0,BOOKS,0.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2016-10-26,52,NaN,POULTRY,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True
3000884,2016-10-26,52,NaN,PREPARED FOODS,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True
3000885,2016-10-26,52,NaN,PRODUCE,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True
3000886,2016-10-26,52,NaN,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True


In [22]:
# data = pd.read_csv('inputs/train.csv', index_col='id')
# data['date'] = pd.to_datetime(data['date'])
data = testy.copy()
data['nat_holiday'] = 0
data['reg_holiday'] = 0
data['loc_holiday'] = 0


# stores = pd.read_csv('inputs/stores.csv', index_col='store_nbr')
stores = stores_new.copy()
# holidays = pd.read_csv('inputs/holidays_events.csv')
# holidays['date'] = pd.to_datetime(holidays['date'])
holidays = pd.read_csv(path + 'holidays_events.csv',
                       usecols=['date', 'type', 'locale', 'locale_name', 'transferred'],
                       dtype={'type': 'category', 'locale': 'category', 'locale_name': 'category', 'transferred': 'bool'},
                       parse_dates=['date'], infer_datetime_format=True
                       )

# holidays.date = holidays.date.dt.to_period('D')
holidays = holidays.reset_index()
stores = stores.reset_index()

for index, row in holidays.iterrows():
    store_nbr_list = []
    if row['locale'] == 'Local':
        for sindex, srow in stores.iterrows():
            if srow['city'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'loc_holiday'] = 1
    if row['locale'] == 'Regional':
        for sindex, srow in stores.iterrows():
            if srow['state'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'reg_holiday'] = 1
    if row['locale'] == 'National':
        data.loc[data['date'] == row['date'], 'nat_holiday'] = 1

In [23]:
# Get rid of nans
data.transactions.fillna(0, inplace=True)
data['ma_oil'].fillna(method='ffill', inplace=True)

In [24]:
data

,date,store_nbr,transactions,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,ma_oil,wd,nat_holiday,reg_holiday,loc_holiday
0,2013-01-01,25,770.0,AUTOMOTIVE,0.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
1,2013-01-01,25,770.0,BABY CARE,0.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
2,2013-01-01,25,770.0,BEAUTY,2.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
3,2013-01-01,25,770.0,BEVERAGES,810.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
4,2013-01-01,25,770.0,BOOKS,0.0,0.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2016-10-26,52,0.0,POULTRY,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True,0,0,0
3000884,2016-10-26,52,0.0,PREPARED FOODS,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True,0,0,0
3000885,2016-10-26,52,0.0,PRODUCE,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True,0,0,0
3000886,2016-10-26,52,0.0,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,Manta,Manabi,A,11,48.75,50.17,True,0,0,0


In [25]:
data['date'] = pd.to_datetime(data['date'])
pd.to_datetime('2017-04-01') in data.date.unique()

True

In [26]:
# set the range of data used in training
sdate = '2017-04-01'
edate = '2017-08-15'

data['date'] = pd.to_datetime(data['date'])
data_cut_dates = data.loc[(data['date'] >= sdate) & (data['date'] <= edate)]
data_cut_dates

,date,store_nbr,transactions,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,ma_oil,wd,nat_holiday,reg_holiday,loc_holiday
2633796,2017-04-01,25,942.0,AUTOMOTIVE,4.000000,0.0,Salinas,Santa Elena,D,1,NaN,48.570000,False,0,0,0
2633797,2017-04-01,25,942.0,BABY CARE,0.000000,0.0,Salinas,Santa Elena,D,1,NaN,48.570000,False,0,0,0
2633798,2017-04-01,25,942.0,BEAUTY,4.000000,0.0,Salinas,Santa Elena,D,1,NaN,48.570000,False,0,0,0
2633799,2017-04-01,25,942.0,BEVERAGES,2191.000000,7.0,Salinas,Santa Elena,D,1,NaN,48.570000,False,0,0,0
2633800,2017-04-01,25,942.0,BOOKS,0.000000,0.0,Salinas,Santa Elena,D,1,NaN,48.570000,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877925,2017-08-15,52,2255.0,POULTRY,397.339996,0.0,Manta,Manabi,A,11,47.57,48.648571,True,0,0,0
2877926,2017-08-15,52,2255.0,PREPARED FOODS,104.417999,0.0,Manta,Manabi,A,11,47.57,48.648571,True,0,0,0
2877927,2017-08-15,52,2255.0,PRODUCE,3803.367920,8.0,Manta,Manabi,A,11,47.57,48.648571,True,0,0,0
2877928,2017-08-15,52,2255.0,SCHOOL AND OFFICE SUPPLIES,17.000000,0.0,Manta,Manabi,A,11,47.57,48.648571,True,0,0,0


In [27]:
# Train data
X = np.zeros((data_cut_dates.family.nunique(), data_cut_dates.date.nunique(), 4))
print(f'x shape: {X.shape}')

# Truth data
y_truth = np.zeros(X.shape[:2])
print(f'y shape: {y_truth.shape}')

family_groups = data_cut_dates.groupby('family')
for i, (family, family_group) in enumerate(family_groups):
    for j, (date, group) in enumerate(family_group.groupby('date')):
        if i == 1 and j == 1:
            first = False
        X[i][j][0] = group.onpromotion.sum()
        X[i][j][1] = group.ma_oil.mean()
        X[i][j][2] = group.wd.any()
        X[i][j][3] = min(group.nat_holiday.sum(), 1)
        y_truth[i][j] = group.sales.sum()

x shape: (33, 137, 4)
y shape: (33, 137)


In [29]:
from lightgbm import LGBMRegressor

#Can add weights/grouping to this as well if we want
def RMSLE(y, y_pred):
    is_higher_better = False
    loss = 0
    for i in range(len(y)):
        loss += (Math.log(1 + y_pred[i]) - Math.log(1 + y[i]))**2
    loss = loss / len(y)
    return "RMSLE", loss, is_higher_better

y_pred = np.zeros(X.shape[:2])
for i, (x, y) in enumerate(zip(X, y_truth)):
    model = LGBMRegressor(n_estimators=100)
    model.fit(x, y, eval_metric=RMSLE)
    # y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)
    y_pred[i] = model.predict(x)
    y_pred[i][np.where(y_pred[i] < 0)] = 0

In [30]:
loss = np.sum( ( np.log(y_pred + 1) + np.log(y_truth + 1) )**2, axis=1 ) / y_truth.shape[1]

In [31]:
loss

array([143.00062395,  22.27301756, 128.6824234 , 591.01599794,
         8.77860002, 423.7241348 , 173.1703009 , 495.91368356,
       467.08562353, 379.34267142, 343.92188635, 314.91123765,
       618.73151919, 215.08736077,  77.13621223, 220.26418174,
       219.37698258,  45.91933033, 375.28805337, 164.85199417,
       183.75312135, 142.15550256, 282.16594184, 136.88150469,
       391.52212966, 379.12993888, 151.90974696, 165.95568073,
       394.85295453, 289.5796969 , 552.51133125, 137.90236144,
       197.84043666])

In [48]:
X_test = np.zeros((data_cut_dates.family.nunique(), 15))

In [ ]:
# Create submission

df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred.values
df_sub.to_csv('submission.csv', index=True)

In [47]:
for i , (family, _) in enumerate(data_cut_dates.groupby('family')):
    for j in range(15):
        collection = data.loc[(pd.to_datetime(f'2017-08-15') == data.date) & (family == data.family)]
        X_test[i][j][0] = 
        X_test[i][j][1] = collection.ma_oil.mean()
        
        X[i][j][0] = group.onpromotion.sum()
        X[i][j][1] = group.ma_oil.mean()
        X[i][j][2] = group.wd.any()
        X[i][j][3] = min(group.nat_holiday.sum(), 1)

,date,store_nbr,transactions,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,ma_oil,wd,nat_holiday,reg_holiday,loc_holiday
2876148,2017-08-15,25,849.0,AUTOMOTIVE,7.0,0.0,Salinas,Santa Elena,D,1,47.57,48.648571,True,0,0,0
2876181,2017-08-15,1,1693.0,AUTOMOTIVE,4.0,0.0,Quito,Pichincha,D,13,47.57,48.648571,True,0,0,0
2876214,2017-08-15,2,1737.0,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,47.57,48.648571,True,0,0,0
2876247,2017-08-15,3,2956.0,AUTOMOTIVE,12.0,0.0,Quito,Pichincha,D,8,47.57,48.648571,True,0,0,0
2876280,2017-08-15,4,1283.0,AUTOMOTIVE,7.0,0.0,Quito,Pichincha,D,9,47.57,48.648571,True,0,0,0
2876313,2017-08-15,5,1310.0,AUTOMOTIVE,9.0,0.0,Santo Domingo,Santo Domingo de los Tsachilas,D,4,47.57,48.648571,True,0,0,0
2876346,2017-08-15,6,1589.0,AUTOMOTIVE,7.0,0.0,Quito,Pichincha,D,13,47.57,48.648571,True,0,0,0
2876379,2017-08-15,7,1780.0,AUTOMOTIVE,5.0,0.0,Quito,Pichincha,D,8,47.57,48.648571,True,0,0,0
2876412,2017-08-15,8,2621.0,AUTOMOTIVE,4.0,0.0,Quito,Pichincha,D,8,47.57,48.648571,True,0,0,0
2876445,2017-08-15,9,2155.0,AUTOMOTIVE,15.0,0.0,Quito,Pichincha,B,6,47.57,48.648571,True,0,0,0
